In [394]:
# Imports
import pandas as pd
import numpy as np
import pyodbc
import math


In [395]:
server = 'tcp:sports-iq.database.windows.net,1433' 
database = 'main' 
username = 'CloudSAeab82b26' 
password = 'EkCVer7h44VMnpe'

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [396]:
sql = "SELECT *, FirstName + ' ' + LastName AS [FullName] FROM MLB.Players"

players = pd.read_sql(sql, cnxn)

print(players.shape)

C:\Users\benja\AppData\Local\Temp\ipykernel_26412\955873534.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  players = pd.read_sql(sql, cnxn)


(14774, 13)


In [397]:
seasons = [2018, 2019, 2020, 2021, 2022, 2023, 2024]
all_player_data = pd.DataFrame()

for season in seasons:
	sql = f"""
	SELECT
			p.PlayerID
			,p.FirstName + ' ' + p.LastName AS [FullName]
			,p.[Position]
			,DATEDIFF(hour,p.DateOfBirth,GETDATE())/8766 AS Age
			,CASE WHEN p.PositionCategory = 'P' THEN pit.Games ELSE b.Games END AS [Games]
			, AtBats, Runs, b.Hits, Doubles, Triples, b.HomeRuns, RunsBattedIn AS [RBI], BattingAverage, b.Strikeouts, b.Walks, HitByPitch, Steals, CaughtStealing, b.OBP, Slug, OBPPlus
			,pit.Starts, pit.Saves, pit.Wins, pit.Losses, pit.InningsPitched, pit.ERA, pit.EarnedRuns,pit.StrikeoutsPerNineInnings, pit.WalksPerNineInnings, pit.WHIP, pit.BattingAverageAgainst, pit.Shutouts
			,pit.Hits AS [PitchingHits]
			,pit.HomeRuns AS [PitchingHomeRuns]
			,pit.Strikeouts AS [PitchingStrikeouts]
			,pit.Walks AS [PitchingWalks]
			,pit.OBP AS [PitchingOBP]
	FROM
			MLB.Players p 
			LEFT JOIN MLB.Batting b ON b.PlayerID = p.PlayerID
					AND b.Season = {season}
			LEFT JOIN MLB.Pitching pit ON pit.PlayerID = p.PlayerID
					AND pit.Season = {season}
	WHERE
			CASE WHEN p.PositionCategory = 'P' THEN pit.Games ELSE b.Games END IS NOT NULL
	"""

	fantasy_map = {}

	player_data = pd.read_sql(sql, cnxn)
	player_ranking = pd.read_csv(f'PlayerRankings-{season}.csv')
	
	missing_players = []
	dup_players = []

	for index, row in player_ranking.iterrows():
		rslt_df = players[players['FullName'] == row.Name]
		if (rslt_df.size == 0):
			missing_players.append(row.Name)
			continue

		if (rslt_df.shape[0] > 1):
			condition1 = rslt_df['Position'] == row.Pos
			condition2 = rslt_df['PositionCategory'] == row.Pos
			rslt_df = rslt_df[condition1 | condition2]

			if (rslt_df.shape[0] != 1):
				dup_players.append(row.Name)
				continue

		playerID = rslt_df.iloc[0]['PlayerID']

		round = math.floor(row.Round / 3)

		if (round == 0):
			round = 1

		fantasy_map[playerID] = (round, row.Rank)
	
	print(len(missing_players))
	print(missing_players)
	print(len(dup_players))
	print(dup_players)

	player_data['FantasyRank'] = player_data.apply(lambda x: fantasy_map[int(x.PlayerID)][1] if int(x.PlayerID) in fantasy_map else -1 , axis=1)
	player_data['FantasyRound'] = player_data.apply(lambda x: fantasy_map[int(x.PlayerID)][0] if int(x.PlayerID) in fantasy_map else -1 , axis=1)

	player_data = player_data[player_data['FantasyRank'] != -1]
	all_player_data = pd.concat([all_player_data, player_data])

print(all_player_data.shape)
print(all_player_data.head())

C:\Users\benja\AppData\Local\Temp\ipykernel_26412\850989667.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  player_data = pd.read_sql(sql, cnxn)


8
['Ronald Acuna', 'Seung Hwan Oh', 'Joseph Wendle', 'Samuel Dyson', 'Jackie Bradley', 'Dee Gordon', 'Daniel Winkler', 'Albert Almora']
3
['Jose Martinez', 'David Hernandez', 'Ryan Braun']


C:\Users\benja\AppData\Local\Temp\ipykernel_26412\850989667.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  player_data = pd.read_sql(sql, cnxn)


15
['Ronald Acuna', 'Peter Alonso', 'Yulieski Gurriel', 'Daniel Santana', 'Thomas Pham', 'Giovanny Urshela', 'Samuel Dyson', 'Jonathan Gray', 'Shohei (H) Ohtani', 'Tom Kahnle', 'Lourdes Gurriel', 'Jackie Bradley', 'Tommy LaStella', 'Jonathon Berti', 'Dee Gordon']
4
['Nelson Cruz', 'Ryan Braun', 'Jacob Webb', 'Jose Martinez']


C:\Users\benja\AppData\Local\Temp\ipykernel_26412\850989667.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  player_data = pd.read_sql(sql, cnxn)


...

6
['Michael Harris', 'Ronald Acuna', 'Luis Robert', 'Carl Edwards', 'Jazz Chisholm', 'Luis V Garcia']
0
[]


C:\Users\benja\AppData\Local\Temp\ipykernel_26412\850989667.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  player_data = pd.read_sql(sql, cnxn)


6
['Ronald Acuna', 'Luis Robert', 'Michael Harris', 'Jazz Chisholm', 'Luis V Garcia', 'LaMonte Wade Jr']
0
[]


C:\Users\benja\AppData\Local\Temp\ipykernel_26412\850989667.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  player_data = pd.read_sql(sql, cnxn)


4
['Jazz Chisholm', 'Luis V Garcia', 'Michael Harris', 'Luis Robert']
0
[]
(2871, 40)
    PlayerID          FullName Position  Age  Games  AtBats  Runs   Hits  \
3       4289    Patrick Corbin       SP   35     33    61.0   1.0   12.0   
5       4292      Oliver Perez       RP   43     51     NaN   NaN    NaN   
7       4295        Nick Ahmed       SS   34    153   516.0  61.0  121.0   
10      4297  Paul Goldschmidt       1B   37    158   593.0  95.0  172.0   
14      4301    Ender Inciarte       CF   34    156   597.0  83.0  158.0   

    Doubles  Triples  ...   WHIP  BattingAverageAgainst  Shutouts  \
3       1.0      0.0  ...  1.050                  0.203       1.0   
5       NaN      NaN  ...  0.742                  0.142       0.0   
7      33.0      5.0  ...    NaN                    NaN       NaN   
10     35.0      5.0  ...    NaN                    NaN       NaN   
14     27.0      6.0  ...    NaN                    NaN       NaN   

    PitchingHits  PitchingHomeRuns  Pitchi

In [398]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [399]:
# Preprocessing
features = ['Position', 'Age', 'Games', 'AtBats', 'Runs', 'Hits', 'Doubles', 'Triples', 'HomeRuns', 'RBI', 'BattingAverage', 'Strikeouts', 'Walks', 'HitByPitch', 'Steals', 'CaughtStealing',
						'OBP', 'Slug', 'OBPPlus', 'Starts', 'Saves', 'Wins', 'Losses', 'InningsPitched', 'ERA', 'EarnedRuns', 'StrikeoutsPerNineInnings', 'WalksPerNineInnings', 'WHIP', 'BattingAverageAgainst',
						'Shutouts', 'PitchingHits', 'PitchingHomeRuns', 'PitchingStrikeouts', 'PitchingWalks', 'PitchingOBP']


#Attempt Batters only
batters = all_player_data[np.isnan(all_player_data['InningsPitched'])]

X, y = batters[features], batters['FantasyRound']

print(X.shape)

(1604, 36)


In [400]:
# Encode Position

X.Position=X.Position.astype('category').cat.codes

X = X.fillna(0)

C:\Users\benja\AppData\Local\Temp\ipykernel_26412\1158127870.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.Position=X.Position.astype('category').cat.codes


In [401]:
# MinMaxScaler

feature_scaler = MinMaxScaler()
X = feature_scaler.fit_transform(X)

# print(X)

In [402]:
# 4 Split the data into Train and Test so we can predict
TEST_PROP, RANDOM_SEED = 0.5, 0
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_PROP, random_state=RANDOM_SEED)

In [403]:
model =  RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [404]:
# 6 Evaluation: Test the model via predicting
y_pred = model.predict(X_train)
acc_train = accuracy_score(y_train, y_pred)
print("accuracy score (training data): %.2f" % acc_train)
y_pred = model.predict(X_test)
acc_test = accuracy_score(y_test, y_pred)
print("accuracy_score (test data): %.2f" % acc_test)

accuracy score (training data): 1.00
accuracy_score (test data): 0.25
